In [1]:
#
import numpy as np
import os, sys
from astropy.table import Table, vstack
import bokeh.plotting as bk
from bokeh.models import Tabs
from bokeh.io import show

from main import html_postages

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

In [2]:
#load LS dr8 SWEEPS sample within a GAMA field footprint
dr8 = np.load('/global/cscratch1/sd/qmxp55/bgstargets_output/dr8-south_sweep_211.5_223.5_-2.0_3.0.npy')
#get BGS targets
bgs_any = ((dr8['BGSBITS'] & 2**(20)) != 0)

## Basic example: Gallery of very bright galaxies

In [3]:
#
dr = 'dr8'
survey = 'south'
veto = {'BGS':(bgs_any),'Not BGS':(~bgs_any)}
info_list = ['RA', 'DEC', 'RMAG', 'GMAG', 'ZMAG', 'TYPE']
info = {key:dr8[key] for key in info_list}
#info = {'RA':dr8['RA'], 'DEC':dr8['DEC'],'r':dr8['RMAG'],'g':dr8['GMAG'],'z':dr8['ZMAG'], 'morph':dr8['TYPE']}
layer_list = ['%s-%s' %(dr, survey), '%s-%s-model' %(dr, survey), '%s-%s-resid' %(dr, survey)]
coord = [dr8['RA'], dr8['DEC']]
idx = list(np.where((bgs_any) & (dr8['RMAG'] < 16)))[0]
title = 'Title of gallery'
main_text = 'Main text to describe gallery'
buttons_text = 'select which images and object \n markers you want to display'
grid = [4,4]
savefile = 'example_gallery'

#
html_postages(coord=coord, idx=idx, veto=veto, info=info, grid=grid, layer_list=layer_list, title=title, 
              main_text=main_text, buttons_text=buttons_text, savefile=savefile)


Loading BokehJS ...

example_gallery.html
32.0 122


In [4]:
#
coord = [dr8['RA'], dr8['DEC']]
idx = list(np.where((bgs_any) & (dr8['RMAG'] < 17)))[0]
layer_list = ['%s-%s' %(dr, survey), '%s-%s-model' %(dr, survey), '%s-%s-resid' %(dr, survey)]

html_postages(coord=coord, idx=idx, layer_list=layer_list)

Loading BokehJS ...

32.0 122


## Example with two catalogues

For this example, we will compare targets of LS DR8 with DR9j by creating a common catalogue only with needed parameters for speed purposes. Gallery is centred ad BGS objects in either with rmag < 18.

In [5]:
#

#load dr9j data
dr9 = np.load('/global/cscratch1/sd/qmxp55/bgstargets_output/dr9j-south_sweep_211.5_223.5_-2.0_3.0.npy')

#create common catalogue
def merge_cats(cat1, cat2, cols):
    
    tab1 = Table()
    tab2 = Table()
    
    cat1_val = list(cat1.values())[0]
    cat1_keys = list(cat1.keys())[0]
    
    cat2_val = list(cat2.values())[0]
    cat2_keys = list(cat2.keys())[0]

    #catalogue 1
    for col in cols:
        tab1[col] = cat1_val[col]
    tab1['BGSBITS'] = cat1_val['BGSBITS']
    tab1[cat1_keys] = np.ones_like(cat1_val, dtype=bool)
    tab1[cat2_keys] = np.zeros_like(cat1_val, dtype=bool)
    
    #catalogue 2
    for col in cols:
        tab2[col] = cat2_val[col]
    tab2['BGSBITS'] = cat2_val['BGSBITS']
    tab2[cat1_keys] = np.zeros_like(cat2_val, dtype=bool)
    tab2[cat2_keys] = np.ones_like(cat2_val, dtype=bool)
    
    #stacking catalogues
    tab = vstack([tab1, tab2])
    
    return tab

tab = merge_cats(cat1={'dr8':dr8}, cat2={'dr9j':dr9}, cols=info_list)


In [6]:
#

bgs_tab = ((tab['BGSBITS'] & 2**(20)) != 0)

#
#dr = 'dr8'
survey = 'south'
veto = {'DR8':tab['dr8'],'DR9j':tab['dr9j']}
info = {key:tab[key] for key in info_list}
layer_list = ['%s-%s' %('dr8', survey), '%s-%s-model' %('dr8', survey), '%s-%s-resid' %('dr8', survey),
              '%s-%s' %('dr9j', survey), '%s-%s-model' %('dr9j', survey), '%s-%s-resid' %('dr9j', survey)]
coord = [tab['RA'], tab['DEC']]
idx = list(np.where((bgs_tab) & (tab['RMAG'] < 18)))[0]
title = 'Comparisson of DR8 with DR9j targets'
main_text = 'For this example, we will compare targets of LS DR8 with DR9j by creating a common catalogue only with needed parameters for speed purposes. Gallery is centred ad BGS objects in either with rmag < 18.'
buttons_text = 'select which images and object \n markers you want to display'
grid = [2,4]
savefile = 'example_gallery_comparisson'

#
html_postages(coord=coord, idx=idx, veto=veto, info=info, grid=grid, layer_list=layer_list, title=title, 
              main_text=main_text, buttons_text=buttons_text, savefile=savefile)

Loading BokehJS ...

example_gallery_comparisson.html
32.0 122


## Example with multiple galleries in a single html file

We will create various set of galleries for different magnitude bins and put them in a single webpage divided by tabs.

In [7]:
#
tabs = []

#
dr = 'dr8'
survey = 'south'
veto = {'BGS':(bgs_any),'Not BGS':(~bgs_any)}
info_list = ['RA', 'DEC', 'RMAG', 'GMAG', 'ZMAG', 'TYPE']
info = {key:dr8[key] for key in info_list}
#info = {'RA':dr8['RA'], 'DEC':dr8['DEC'],'r':dr8['RMAG'],'g':dr8['GMAG'],'z':dr8['ZMAG'], 'morph':dr8['TYPE']}
layer_list = ['%s-%s' %(dr, survey), '%s-%s-model' %(dr, survey), '%s-%s-resid' %(dr, survey)]
coord = [dr8['RA'], dr8['DEC']]
grid = [4,4]

N = np.linspace(14, 20, 7)

for i in range(len(N[:-1])):
    
    idx = list(np.where((bgs_any) & (dr8['RMAG'] < N[i+1]) & (dr8['RMAG'] > N[i])))[0]
    title = 'BGS targets within %i < rmag < %i' %(N[i], N[i+1])
    main_text = 'This gallery shows %i of %i available BGS targets within rmag bin.' %(grid[0]*grid[1], len(idx))
    buttons_text = 'select which images and object \n markers you want to display'
    tab_title = '%i < rmag < %i' %(N[i], N[i+1])

#
    tab = html_postages(coord=coord, idx=idx, veto=veto, info=info, grid=grid, layer_list=layer_list, title=title, 
              main_text=main_text, buttons_text=buttons_text, tab=True, tab_title=tab_title)


    tabs.append(tab)
    

Loading BokehJS ...

32.0 122


Loading BokehJS ...

32.0 122


Loading BokehJS ...

...

32.0 122


Loading BokehJS ...

32.0 122


Loading BokehJS ...

32.0 122


In [8]:
#
savefile = 'example_gallery_multiple_tabs'

bk.output_notebook()
if savefile is not None:
    html_page = savefile + '.html'
    bk.output_file(html_page, title=savefile)
    print(html_page)

# Put all the tabs into one application
page = Tabs(tabs=tabs)

# Put the tabs in the current document for display
show(page)

Loading BokehJS ...

example_gallery_multiple_tabs.html
